# docker-compose

## コンテナ設定リスト

|コンテナ|サービス|VIP|port|
|---|---|---|---|
|mysql5.6|mysql|172.17.33.11|3306|
|mysql5.6|ssh|172.17.33.11|2222|
|openldap|ldap|172.17.33.12|389|
|openldap|ssh|172.17.33.12|2223|
|mail_server|imap|172.17.33.13|1143|
|mail_server|smtp|172.17.33.13|2525|
|mail_server|ssh|172.17.33.13|2224|
|mail_server_virtual_for joruri|imap|172.17.33.14|1144|
|mail_server_virtual_for joruri|smtp|172.17.33.14|2526|
|mail_server_virtual_for joruri|ssh|172.17.33.14|2224|

## 現在のステータス

### イメージリスト

In [6]:
import pandas as pd
import subprocess
import sys

### コマンドの実行結果を取得する
res = subprocess.run(["docker", "images"], stdout=subprocess.PIPE)

### バイト列を文字列にする
a = res.stdout.decode('utf-8')
#print(a)

### 改行を区切りに要素に入れる
b = a.split("\n")
#print(b)

### それぞれの要素で半角空白2文字を区切り文字にし、空の要素は除く
c = []
for i in range(len(b)):
    item = b[i]
    if i != 0:
        c.insert(0,tuple(filter(lambda x: x != '', item.split("  "))))
#print(c)
        
### 列名をリストcolに入れる
col = list(filter(lambda x: x != '', b[0].split("  ")))

### データをデータフレームに入れて出力する
del c[0]
df = pd.DataFrame(c, columns=col)
df

,REPOSITORY,TAG,IMAGE ID,CREATED,SIZE
0,ftakao2007/mysql5.6_centos7,latest,09a773ad8e34,6 days ago,2.16GB
1,ftakao2007/openldap_centos7,latest,572b38993fa4,4 days ago,596MB
2,ftakao2007/mail_centos7,latest,0733ee002c86,2 days ago,656MB
3,ftakao2007/mail_centos7,virtual_for_jorurimail,35e0eebeeb79,2 days ago,657MB


### コンテナリスト

In [25]:
import pandas as pd
import subprocess
import sys

### コマンドの実行結果を取得する
res = subprocess.run(['docker', 'ps', '-a'], stdout=subprocess.PIPE)

### バイト列を文字列にする
a = res.stdout.decode('utf-8')
#print(a)

### 改行を区切りに要素に入れる
b = a.split("\n")
#print(b)

### それぞれの要素で半角空白2文字を区切り文字にし、空の要素は除く
c = []
for i in range(len(b)):
    item = b[i]
    if i != 0:
        c.insert(0,tuple(filter(lambda x: x != '', item.split("  "))))
#print(c)
        
### 列名をリストcolに入れる
col = list(filter(lambda x: x != '', b[0].split("  ")))

### データをデータフレームに入れて出力する
del c[0]
df = pd.DataFrame(c, columns=col)
df

### COMMAND,CREATED,STATUS列を削除する
del_command = []
for i in c:
    i_tmp = list(i)
    del i_tmp[2:5]
    del_command.append(tuple(i_tmp))
del col[2:5]
df = pd.DataFrame(del_command, columns=col)
df

,CONTAINER ID,IMAGE,PORTS,NAMES
0,c91b49b1012f,ftakao2007/mysql5.6_centos7:latest,"0.0.0.0:3306->3306/tcp, 0.0.0.0:2222->22/tcp",mysql5.6_centos7
1,49914008c382,ftakao2007/openldap_centos7:latest,"0.0.0.0:389->389/tcp, 0.0.0.0:2223->22/tcp",openldap_centos7
2,d8eb45ab72c7,ftakao2007/mail_centos7:latest,"0.0.0.0:2224->22/tcp, 0.0.0.0:2525->25/tcp, 0...",mail_server_centos7
3,01faf6b40cba,ftakao2007/mail_centos7:virtual_for_jorurimail,"0.0.0.0:2225->22/tcp, 0.0.0.0:2526->25/tcp, 0...",mail_server_virtual_for_jorurimail_centos7


### コンテナリスト(複数ホスト指定可能, ユーザに必要な情報を表示、PORTSですべて表示)

In [30]:
import pandas as pd
import subprocess
import sys
pd.set_option("display.max_colwidth", 100)

### 全てのdockerhost
dockerhost_list = ['10.136.3.168','10.136.3.180']

result = []

for j in range(len(dockerhost_list)):
    ### コマンドの実行結果を取得する
    res = subprocess.run(['ssh',dockerhost_list[j],'docker', 'ps'], stdout=subprocess.PIPE)

    ### バイト列を文字列にする
    a = res.stdout.decode('utf-8')
    #print(a)

    ### 改行を区切りに要素に入れる
    b = a.split("\n")
    #print(b)

    ### それぞれの要素で半角空白2文字を区切り文字にし、空の要素は除く
    c = []
    for i in range(len(b)):
        item = b[i]
        if i != 0:
            c.insert(0,tuple(filter(lambda x: x != '', item.split("  "))))

    ### 列名の要素を削除
    del c[0]
    #print(c)

    ### 列名をリストcolに入れる
    col = list(filter(lambda x: x != '', b[0].split("  ")))

    ### CONTAINER ID, IMAGE, COMMAND,CREATED,STATUS列を削除し、DOCKERHOST列を追加する
    for i in c:
        i_tmp = list(i)
        del i_tmp[0:5]
        i_tmp.insert(0,dockerhost_list[j])
        i_tmp = list(map(lambda x: x.replace('0.0.0.0:','') , i_tmp))
        result.append(tuple(i_tmp))
    del col[0:5]
    col.insert(0,'DOCKER HOST')

### 表示
df = pd.DataFrame(result, columns=col)
df

,DOCKER HOST,PORTS,NAMES
0,10.136.3.168,"3306->3306/tcp, 2222->22/tcp",mysql5.6_centos7
1,10.136.3.168,"389->389/tcp, 2223->22/tcp",openldap_centos7
2,10.136.3.168,"2224->22/tcp, 2525->25/tcp, 1143->143/tcp",mail_server_centos7
3,10.136.3.168,"2225->22/tcp, 2526->25/tcp, 1144->143/tcp",mail_server_virtual_for_jorurimail_centos7
4,10.136.3.180,"3306/tcp, 2292->22/tcp, 8095->80/tcp",redmine2.6.10_centos7
5,10.136.3.180,"5902->5902/tcp, 2244->22/tcp",gui_ubuntu16.04
6,10.136.3.180,"3000->3000/tcp, 8888->8888/tcp, 2238->22/tcp",jupyter_iruby


In [34]:
%%bash
### mysql接続確認
mysql -ujoruri -p'pass' -h 10.136.3.168 -P 3306 -e "SELECT CURDATE();" 2> /dev/null

CURDATE()
2017-07-20


### MySQLに登録

In [38]:
import MySQLdb
import pandas.io.sql as mysql

### DB接続
#con = MySQLdb.connect(database='webmail', host='3306', user='jupyter', passwd='pass', charset='utf8')
con = MySQLdb.connect(host='10.136.3.168', user='jupyter', passwd='pass', charset='utf8')
sql = "show databases"

### pandasのDataFrameの形でデータを取り出す
df = mysql.read_sql(sql, con) 
con.close()
df.head(5)

,Database
0,information_schema
1,jorurimail
2,mysql
3,performance_schema
4,sampledb


## (以下作業領域)

In [24]:
import subprocess
args = ['docker', 'ps', '-a']
try:
    res = subprocess.check_output(args)
    print(res)
except:
    print("Error.")

b'CONTAINER ID        IMAGE                                            COMMAND                  CREATED             STATUS              PORTS                                                               NAMES\n01faf6b40cba        ftakao2007/mail_centos7:virtual_for_jorurimail   "sh -c \'exec /usr/..."   2 days ago          Up 2 days           0.0.0.0:2225->22/tcp, 0.0.0.0:2526->25/tcp, 0.0.0.0:1144->143/tcp   mail_server_virtual_for_jorurimail_centos7\nd8eb45ab72c7        ftakao2007/mail_centos7:latest                   "sh -c \'exec /usr/..."   2 days ago          Up 2 days           0.0.0.0:2224->22/tcp, 0.0.0.0:2525->25/tcp, 0.0.0.0:1143->143/tcp   mail_server_centos7\n49914008c382        ftakao2007/openldap_centos7:latest               "sh -c \'exec /usr/..."   2 days ago          Up 2 days           0.0.0.0:389->389/tcp, 0.0.0.0:2223->22/tcp                          openldap_centos7\nc91b49b1012f        ftakao2007/mysql5.6_centos7:latest               "sh -c \'exec /usr/..."   2 

In [14]:
%%bash
docker images

REPOSITORY                    TAG                      IMAGE ID            CREATED             SIZE
ftakao2007/mail_centos7       virtual_for_jorurimail   35e0eebeeb79        18 hours ago        657MB
ftakao2007/mail_centos7       latest                   0733ee002c86        19 hours ago        656MB
ftakao2007/openldap_centos7   latest                   572b38993fa4        2 days ago          596MB
ftakao2007/mysql5.6_centos7   latest                   09a773ad8e34        4 days ago          2.16GB
ftakao2007/centos7            latest                   0fede72fa1fe        4 days ago          488MB


In [15]:
%%bash
docker ps -a

CONTAINER ID        IMAGE                                            COMMAND                  CREATED             STATUS              PORTS                                                               NAMES
d466248a93d9        ftakao2007/mail_centos7:virtual_for_jorurimail   "sh -c 'exec /usr/..."   6 minutes ago       Up 6 minutes        0.0.0.0:2225->22/tcp, 0.0.0.0:2526->25/tcp, 0.0.0.0:1144->143/tcp   mail_server_virtual_for_jorurimail_centos7
8784dfcb21c3        ftakao2007/mail_centos7:latest                   "sh -c 'exec /usr/..."   22 minutes ago      Up 22 minutes       0.0.0.0:2224->22/tcp, 0.0.0.0:2525->25/tcp, 0.0.0.0:1143->143/tcp   mail_server_centos7
3bc5d977b203        ftakao2007/openldap_centos7:latest               "sh -c 'exec /usr/..."   26 minutes ago      Up 26 minutes       0.0.0.0:389->389/tcp, 0.0.0.0:2223->22/tcp                          openldap_centos7
4c0ae849ad6a        ftakao2007/mysql5.6_centos7:latest               "sh -c 'exec /usr/..."   29 minutes a

In [ ]:
import pandas as pd
import subprocess
import sys

### コマンドの実行結果を取得する
res = subprocess.run(["docker", "images"], stdout=subprocess.PIPE)

### バイト列を文字列にする
a = res.stdout.decode('utf-8')
#print(a)

### 改行を区切りに要素に入れる
b = a.split("\n")
#print(b)

### それぞれの要素で半角空白2文字を区切り文字にし、空の要素は除く
c = []
for i in range(len(b)):
    item = b[i]
    if i != 0:
        c.insert(0,tuple(filter(lambda x: x != '', item.split("  "))))
#print(c)

pd_index = []
for i in c[1:]:
    pd_index.append(i[2])
print(pd_index)
        
### 列名をリストcolに入れる
col = list(filter(lambda x: x != '', b[0].split("  ")))

### データをデータフレームに入れて出力する
df = pd.DataFrame(c[1:], columns=col, index=pd_index)
df